# Maxwell Model: Stress Relaxation Fitting

## What is This?

This notebook introduces **RheoJAX** through the **simplest viscoelastic model**: the Maxwell model. The Maxwell element combines a spring and dashpot in series, providing the foundation for understanding all linear viscoelastic behavior.

**Why Maxwell?** It captures the essence of viscoelasticity with only 2 parameters and analytical solutions, making it the perfect gateway to modern rheological analysis.

## The Maxwell Model

**Constitutive Equation (Differential Form):**
$$\sigma + \frac{\eta}{G}\dot{\sigma} = \eta\dot{\gamma}$$

**Relaxation Modulus:**
$$G(t) = G \exp\left(-\frac{t}{\tau}\right), \quad \tau = \frac{\eta}{G}$$

**Storage and Loss Moduli (Oscillatory):**
$$G'(\omega) = G\frac{(\omega\tau)^2}{1 + (\omega\tau)^2}, \quad G''(\omega) = G\frac{\omega\tau}{1 + (\omega\tau)^2}$$

where:
- $G$ = spring modulus (Pa) — instantaneous elastic response
- $\eta$ = dashpot viscosity (Pa·s) — resistance to flow
- $\tau = \eta/G$ = relaxation time (s) — characteristic timescale

**Physical Meaning:**
- At short times/high frequencies: elastic solid-like ($G' \to G$)
- At long times/low frequencies: viscous liquid-like ($G' \sim \omega^2$, $G'' \sim \omega$)
- The relaxation time $\tau$ separates these two regimes

> **Handbook:** See [Maxwell Model](../../docs/source/models/classical/maxwell.rst) for comprehensive theory, [Classical Models Overview](../../docs/source/models/classical/index.rst) for model selection guidance, and Ferry (1980) *Viscoelastic Properties of Polymers* for canonical reference.

## Learning Objectives

After completing this notebook, you will be able to:
- Fit the Maxwell model using both Pipeline API and Modular API approaches
- Leverage NLSQ optimization for 5-270x speedup over SciPy
- Perform Bayesian inference with NLSQ→NUTS warm-start workflow
- Interpret all 6 ArviZ diagnostic plots for MCMC convergence
- Extract physically meaningful parameters with uncertainty quantification

## Prerequisites

Basic understanding of:
- Rheological concepts (stress, strain, relaxation)
- Linear viscoelasticity
- Python and NumPy

**Estimated Time:** 30-40 minutes

In [ ]:
# Google Colab Setup - Run this cell first!
# Skip if running locally with rheojax already installed

import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # Install rheojax and dependencies
    !pip install -q rheojax
    
    # Colab uses float32 by default - we need float64 for numerical stability
    # This MUST be set before importing JAX
    import os
    os.environ['JAX_ENABLE_X64'] = 'true'
    
    print("✓ RheoJAX installed successfully!")
    print("✓ Float64 precision enabled")

## Setup and Imports

We start by importing necessary libraries. Note the **safe JAX import pattern** - this is critical for ensuring float64 precision throughout the entire JAX stack.

In [ ]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

# Standard scientific computing imports
import time
import warnings

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from rheojax.models import Maxwell

from rheojax.core.data import RheoData
from rheojax.core.jax_config import safe_import_jax, verify_float64

# Rheo imports - always explicit
from rheojax.pipeline.base import Pipeline
from rheojax.pipeline.bayesian import BayesianPipeline

# Safe JAX import - REQUIRED for all notebooks using JAX
# This pattern ensures float64 precision enforcement throughout
jax, jnp = safe_import_jax()

# Verify float64 is enabled (educational demonstration)
verify_float64()
print(f"✓ JAX float64 precision enabled (default dtype bits: {jax.config.jax_default_dtype_bits})")

# Set reproducible random seed
np.random.seed(42)

# Configure matplotlib for publication-quality plots
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

# Suppress warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

import sys, os
sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    plot_nlsq_fit, display_arviz_diagnostics, plot_posterior_predictive
)

FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"


## Maxwell Model Theory

The Maxwell model represents a viscoelastic material as a spring and dashpot in series:

$$G(t) = G_0 \exp\left(-\frac{t}{\tau}\right)$$

where:
- $G(t)$ = relaxation modulus (Pa)
- $G_0$ = initial modulus (Pa) - represents elastic response
- $\tau = \eta / G_0$ = relaxation time (s) - characterizes viscous response
- $\eta$ = viscosity (Pa·s)

**Physical Interpretation:**
- **$G_0$**: Instantaneous elastic modulus - material stiffness at $t=0$
- **$\eta$**: Viscosity - resistance to flow
- **$\tau$**: Time scale for stress relaxation - larger $\tau$ means slower relaxation

**Applicability:**
- Simple polymer melts and solutions
- Materials with single dominant relaxation time
- Limited to small strains (linear viscoelastic regime)

## Generate Synthetic Relaxation Data

We create synthetic stress relaxation data with known parameters to validate our fitting workflow. This allows us to verify numerical accuracy by comparing fitted parameters to true values.

In [ ]:
# True Maxwell parameters
G0_true = 1e5  # Pa
eta_true = 1e3  # Pa·s
tau_true = eta_true / G0_true  # s

print(f"True Parameters:")
print(f"  G0  = {G0_true:.2e} Pa")
print(f"  eta = {eta_true:.2e} Pa·s")
print(f"  tau = {tau_true:.4f} s")

# Generate time array (logarithmically spaced for relaxation)
t = np.logspace(-2, 2, 50)  # 0.01 to 100 seconds

# True relaxation modulus
G_t_true = G0_true * np.exp(-t / tau_true)

# Add realistic Gaussian noise (1-2% relative error)
noise_level = 0.015  # 1.5%
noise = np.random.normal(0, noise_level * G_t_true)
G_t_noisy = G_t_true + noise

print(f"\nData characteristics:")
print(f"  Time range: {t.min():.2f} - {t.max():.2f} s")
print(f"  Number of points: {len(t)}")
print(f"  Noise level: {noise_level*100:.1f}% relative")
print(f"  SNR: {np.mean(G_t_true) / np.std(noise):.1f}")

In [ ]:
# Visualize raw data
fig = plt.figure(figsize=(10, 6))
plt.loglog(t, G_t_noisy, 'o', markersize=6, alpha=0.7, label='Synthetic Data (with noise)')
plt.loglog(t, G_t_true, '--', linewidth=2, alpha=0.5, label='True Maxwell Response')
plt.xlabel('Time (s)', fontsize=12)
plt.ylabel('Relaxation Modulus G(t) (Pa)', fontsize=12)
plt.title('Stress Relaxation Data', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, which='both')
plt.legend(fontsize=11)
plt.tight_layout()
display(fig)
plt.close(fig)


## Approach 1: Pipeline API (Recommended for Standard Workflows)

The **Pipeline API** provides a fluent interface for common analysis tasks. It's ideal for rapid prototyping and standardized workflows.

In [ ]:
# Create RheoData container with metadata
data = RheoData(
    x=t,
    y=G_t_noisy,
    x_units='s',
    y_units='Pa',
    domain='time',
)

# Pipeline API workflow with timing
start_pipeline = time.time()

pipeline = Pipeline(data)
pipeline.fit('maxwell')

pipeline_time = time.time() - start_pipeline

# Extract fitted parameters
model = pipeline.get_last_model()
G0_pipeline = model.parameters.get_value('G0')
eta_pipeline = model.parameters.get_value('eta')
tau_pipeline = eta_pipeline / G0_pipeline

print("\n" + "="*60)
print("PIPELINE API RESULTS")
print("="*60)
print(f"Fitted Parameters:")
print(f"  G0  = {G0_pipeline:.4e} Pa  (true: {G0_true:.4e})")
print(f"  eta = {eta_pipeline:.4e} Pa·s  (true: {eta_true:.4e})")
print(f"  tau = {tau_pipeline:.6f} s  (true: {tau_true:.6f})")
print(f"\nRelative Errors:")
print(f"  G0:  {abs(G0_pipeline - G0_true) / G0_true * 100:.4f}%")
print(f"  eta: {abs(eta_pipeline - eta_true) / eta_true * 100:.4f}%")
print(f"\nOptimization time: {pipeline_time:.4f} s")
print("="*60)

## Approach 2: Modular API (Recommended for Customization)

The **Modular API** provides direct access to model classes with scikit-learn compatible interface. Use this when you need fine control over parameters, bounds, or optimization settings.

In [ ]:
# Create Maxwell model instance
model = Maxwell()

# Set parameter bounds (optional but recommended)
model.parameters.set_bounds('G0', (1e3, 1e7))  # Reasonable modulus range
model.parameters.set_bounds('eta', (1e1, 1e5))  # Reasonable viscosity range

# Fit with timing
start_modular = time.time()

model.fit(t, G_t_noisy)

modular_time = time.time() - start_modular

# Extract fitted parameters
G0_modular = model.parameters.get_value('G0')
eta_modular = model.parameters.get_value('eta')
tau_modular = eta_modular / G0_modular

print("\n" + "="*60)
print("MODULAR API RESULTS")
print("="*60)
print(f"Fitted Parameters:")
print(f"  G0  = {G0_modular:.4e} Pa  (true: {G0_true:.4e})")
print(f"  eta = {eta_modular:.4e} Pa·s  (true: {eta_true:.4e})")
print(f"  tau = {tau_modular:.6f} s  (true: {tau_true:.6f})")
print(f"\nRelative Errors:")
print(f"  G0:  {abs(G0_modular - G0_true) / G0_true * 100:.4f}%")
print(f"  eta: {abs(eta_modular - eta_true) / eta_true * 100:.4f}%")
print(f"\nOptimization time: {modular_time:.4f} s")
print("="*60)

# Verify both approaches give same results
assert np.allclose(G0_pipeline, G0_modular, rtol=1e-2), "Pipeline and Modular should give identical results"
assert np.allclose(eta_pipeline, eta_modular, rtol=1e-2), "Pipeline and Modular should give identical results"
print("\n✓ Pipeline and Modular APIs produce identical results")

## Performance Benchmark: NLSQ vs SciPy

Rheo uses **NLSQ** (GPU-accelerated nonlinear least squares) as the default optimization backend, providing 5-270x speedup over SciPy's `curve_fit`.

The speedup comes from:
1. **JAX JIT compilation** - compiles optimization to optimized XLA code
2. **Automatic differentiation** - exact gradients without numerical approximation
3. **GPU acceleration** - parallel computation on CUDA devices (if available)

Let's measure actual performance on your hardware:

In [ ]:
# Benchmark: Multiple fits to get reliable timing
n_runs = 10
times = []

for i in range(n_runs):
    model_bench = Maxwell()
    start = time.time()
    model_bench.fit(t, G_t_noisy)
    times.append(time.time() - start)

nlsq_mean = np.mean(times[1:])  # Exclude first run (JIT compilation)
nlsq_std = np.std(times[1:])

print("\n" + "="*60)
print("PERFORMANCE BENCHMARK (NLSQ)")
print("="*60)
print(f"Number of runs: {n_runs}")
print(f"First run (with JIT): {times[0]:.4f} s")
print(f"Subsequent runs: {nlsq_mean:.4f} ± {nlsq_std:.4f} s")
print(f"JIT overhead: {times[0] - nlsq_mean:.4f} s")
print(f"\nNOTE: SciPy curve_fit typically takes 0.05-0.5s for this problem")
print(f"Expected speedup: 5-270x depending on problem size and GPU")
print(f"For this small dataset ({len(t)} points), speedup may be modest.")
print(f"Speedup increases dramatically with dataset size (>1000 points).")
print("="*60)

## Results Visualization

We create publication-quality visualizations showing:
1. **Fit quality** - data vs model prediction
2. **Residual analysis** - systematic errors or outliers?

In [ ]:
# Generate predictions
G_t_pred = model.predict(t)
residuals = G_t_noisy - G_t_pred
relative_residuals = residuals / G_t_noisy * 100

# Create figure with subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Left: Fit quality
ax1.loglog(t, G_t_noisy, 'o', markersize=6, alpha=0.7, label='Data', color='#1f77b4')
ax1.loglog(t, G_t_true, '--', linewidth=2, alpha=0.4, label='True', color='gray')
ax1.loglog(t, G_t_pred, '-', linewidth=2.5, label='Fitted', color='#ff7f0e')
ax1.set_xlabel('Time (s)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Relaxation Modulus G(t) (Pa)', fontsize=12, fontweight='bold')
ax1.set_title('Maxwell Model Fit', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3, which='both')
ax1.legend(fontsize=11, framealpha=0.9)

# Right: Residual analysis
ax2.semilogx(t, relative_residuals, 'o', markersize=6, alpha=0.7, color='#2ca02c')
ax2.axhline(0, color='black', linestyle='--', linewidth=1, alpha=0.5)
ax2.axhline(noise_level * 100, color='red', linestyle=':', linewidth=1.5, alpha=0.5, label=f'Expected noise: ±{noise_level*100:.1f}%')
ax2.axhline(-noise_level * 100, color='red', linestyle=':', linewidth=1.5, alpha=0.5)
ax2.set_xlabel('Time (s)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Relative Residuals (%)', fontsize=12, fontweight='bold')
ax2.set_title('Residual Analysis', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend(fontsize=10, framealpha=0.9)

plt.tight_layout()
display(fig)
plt.close(fig)

# Compute fit quality metrics
ss_res = np.sum(residuals**2)
ss_tot = np.sum((G_t_noisy - np.mean(G_t_noisy))**2)
r_squared = 1 - (ss_res / ss_tot)
rmse = np.sqrt(np.mean(residuals**2))

print("\nFit Quality Metrics:")
print(f"  R² = {r_squared:.6f}")
print(f"  RMSE = {rmse:.2e} Pa")
print(f"  Mean |residual| = {np.mean(np.abs(residuals)):.2e} Pa ({np.mean(np.abs(relative_residuals)):.2f}%)")
print(f"  Max |residual| = {np.max(np.abs(residuals)):.2e} Pa ({np.max(np.abs(relative_residuals)):.2f}%)")


## Bayesian Inference: Uncertainty Quantification

While NLSQ provides fast point estimates, **Bayesian inference** quantifies parameter uncertainty through posterior distributions. This is essential when:
- Parameters are poorly constrained by data
- Understanding parameter correlations is important
- Propagating uncertainty to predictions is needed
- Comparing competing models statistically

### Two-Stage Workflow: NLSQ → NUTS

1. **NLSQ optimization** (fast) - find approximate maximum likelihood parameters
2. **NUTS sampling** (slower) - warm-start from NLSQ for 2-5x faster convergence

This warm-start strategy dramatically reduces:
- Number of iterations to convergence
- Divergent transitions (MCMC failures)
- Total computational time

In [ ]:
print("\n" + "="*60)
print("BAYESIAN INFERENCE WITH WARM-START")
print("="*60)
print("Running MCMC sampling... (this may take 1-2 minutes)\n")

# Bayesian inference using warm-start from NLSQ
bayesian_start = time.time()

result = model.fit_bayesian(
    t, G_t_noisy,
    num_warmup=1000,   # Burn-in iterations
    num_samples=2000,  # Posterior samples
    num_chains=1,      # Single chain (faster for demo); use num_chains=4 for production
    initial_values={   # Warm-start from NLSQ
        'G0': model.parameters.get_value('G0'),
        'eta': model.parameters.get_value('eta')
    }
)

bayesian_time = time.time() - bayesian_start

print(f"\nBayesian inference completed in {bayesian_time:.2f} s")
print("="*60)

### Posterior Summary and Convergence Diagnostics

Key metrics for MCMC quality:
- **R-hat < 1.01**: Chains have converged (all parameters must meet this)
- **ESS > 400**: Effective sample size ensures reliable estimates
- **Divergences < 1%**: NUTS sampler is well-behaved

In [ ]:
# Extract posterior samples and diagnostics
posterior = result.posterior_samples
diagnostics = result.diagnostics
summary = result.summary

# Get credible intervals
credible_intervals = model.get_credible_intervals(posterior, credibility=0.95)

print("\n" + "="*60)
print("POSTERIOR SUMMARY")
print("="*60)
print(f"\nParameter Estimates (posterior mean ± std):")
print(f"  G0  = {summary['G0']['mean']:.4e} ± {summary['G0']['std']:.4e} Pa")
print(f"  eta = {summary['eta']['mean']:.4e} ± {summary['eta']['std']:.4e} Pa·s")

print(f"\n95% Credible Intervals:")
print(f"  G0:  [{credible_intervals['G0'][0]:.4e}, {credible_intervals['G0'][1]:.4e}] Pa")
print(f"  eta: [{credible_intervals['eta'][0]:.4e}, {credible_intervals['eta'][1]:.4e}] Pa·s")

print(f"\nConvergence Diagnostics:")
print(f"  R-hat (G0):  {diagnostics['r_hat']['G0']:.4f}  {'✓' if diagnostics['r_hat']['G0'] < 1.01 else '✗ POOR'}")
print(f"  R-hat (eta): {diagnostics['r_hat']['eta']:.4f}  {'✓' if diagnostics['r_hat']['eta'] < 1.01 else '✗ POOR'}")
print(f"  ESS (G0):    {diagnostics['ess']['G0']:.0f}  {'✓' if diagnostics['ess']['G0'] > 400 else '✗ LOW'}")
print(f"  ESS (eta):   {diagnostics['ess']['eta']:.0f}  {'✓' if diagnostics['ess']['eta'] > 400 else '✗ LOW'}")

if 'num_divergences' in diagnostics:
    div_rate = diagnostics['num_divergences'] / result.num_samples * 100
    print(f"  Divergences: {diagnostics['num_divergences']} ({div_rate:.2f}%)  {'✓' if div_rate < 1 else '✗ HIGH'}")

print("\n" + "="*60)

# Check convergence
converged = all([
    diagnostics['r_hat']['G0'] < 1.01,
    diagnostics['r_hat']['eta'] < 1.01,
    diagnostics['ess']['G0'] > 400,
    diagnostics['ess']['eta'] > 400
])

if converged:
    print("\n✓ EXCELLENT CONVERGENCE - All diagnostic criteria met!")
else:
    print("\n⚠ WARNING: Convergence criteria not met. Increase num_warmup or num_samples.")

In [ ]:
# ArviZ diagnostic plots (trace, pair, forest, energy, autocorr, rank)
display_arviz_diagnostics(result, ['G0', 'eta'], fast_mode=FAST_MODE)

## Diagnostic Plot Summary

### Quick Reference Guide

| Plot | Purpose | Good Result | Bad Result | Fix |
|------|---------|-------------|------------|-----|
| **Trace** | Visualize sampling | Fuzzy caterpillar | Trends, jumps | Increase warmup |
| **Pair** | Correlations, divergences | Elliptical, no divergences | Funnel, many divergences | Better priors, reparameterize |
| **Forest** | Credible intervals | Narrow intervals | Very wide | More data, tighter priors |
| **Autocorr** | Mixing quality | Fast decay to 0 | Slow decay | More samples |
| **Rank** | Convergence | Uniform histogram | Non-uniform | More warmup |
| **ESS** | Sample efficiency | ESS > 400 | ESS < 100 | More samples, check mixing |

### Troubleshooting Workflow

If you encounter convergence issues:

1. **Check R-hat and ESS first** (numerical diagnostics)
2. **Rank plot** - most sensitive convergence check
3. **Trace plot** - visual inspection of chains
4. **Autocorr plot** - if ESS is low, check mixing
5. **Pair plot** - if divergences present, check geometry
6. **Forest plot** - assess parameter uncertainty

### Common Issues and Solutions

**High R-hat (> 1.01):**
- Increase `num_warmup` (try 2000 or 5000)
- Use multiple chains (`num_chains=4`)
- Ensure warm-start from NLSQ

**Low ESS (< 400):**
- Increase `num_samples` (try 5000 or 10000)
- Check autocorrelation plot - if high, mixing is poor
- Use warm-start to improve initial proposal

**Many divergences (> 1%):**
- **Critical issue** - results unreliable!
- Warm-start from NLSQ (reduces divergences 10-100x)
- Tighter parameter bounds/priors
- Reparameterize model (e.g., log-transform parameters)

## Physical Interpretation

Let's interpret the fitted parameters in the context of material behavior:

### Parameter Meanings

**Initial Modulus (G₀):**
- Represents instantaneous elastic response
- For our fit: ~1×10⁵ Pa (100 kPa)
- Physical meaning: Material stiffness at t=0
- Typical range: 10² - 10⁹ Pa depending on material

**Viscosity (η):**
- Represents resistance to flow
- For our fit: ~1×10³ Pa·s
- Physical meaning: Controls stress relaxation rate
- Typical range: 10⁻² - 10⁶ Pa·s

**Relaxation Time (τ = η/G₀):**
- Time scale for stress decay to 1/e (~37%) of initial value
- For our fit: ~0.01 s
- Physical meaning: Fast relaxation → fluid-like behavior
- Slow relaxation → solid-like behavior

### Material Classification

Based on relaxation time:
- **τ < 0.1 s**: Predominantly viscous (fluid-like)
- **0.1 < τ < 100 s**: Viscoelastic (mixed behavior)
- **τ > 100 s**: Predominantly elastic (solid-like)

Our material (τ ≈ 0.01 s) exhibits **fluid-like behavior** with rapid stress relaxation.

### Model Limitations

The Maxwell model is valid when:
- ✓ Small strains (linear viscoelastic regime, typically < 10%)
- ✓ Single dominant relaxation time
- ✓ Isothermal conditions

Consider alternative models if:
- ✗ Multiple relaxation times needed → Generalized Maxwell
- ✗ Non-exponential decay → Fractional Maxwell
- ✗ Finite equilibrium modulus → Zener (Standard Linear Solid)
- ✗ Large strain behavior → Nonlinear models (Gent, etc.)

In [ ]:
# Summary table of results
print("\n" + "="*70)
print("FINAL PARAMETER SUMMARY")
print("="*70)
print(f"\n{'Method':<20} {'G0 (Pa)':<15} {'eta (Pa·s)':<15} {'tau (s)':<10}")
print("-"*70)
print(f"{'True Values':<20} {G0_true:<15.4e} {eta_true:<15.4e} {tau_true:<10.6f}")
print(f"{'NLSQ (Point)':<20} {G0_modular:<15.4e} {eta_modular:<15.4e} {tau_modular:<10.6f}")
print(f"{'Bayesian (Mean)':<20} {summary['G0']['mean']:<15.4e} {summary['eta']['mean']:<15.4e} {summary['eta']['mean']/summary['G0']['mean']:<10.6f}")
print("-"*70)

# Uncertainty from Bayesian inference
print(f"\n{'Bayesian Uncertainty (1σ):':<20} {summary['G0']['std']:<15.4e} {summary['eta']['std']:<15.4e}")
print(f"{'Relative Uncertainty:':<20} {summary['G0']['std']/summary['G0']['mean']*100:<15.2f}% {summary['eta']['std']/summary['eta']['mean']*100:<15.2f}%")
print("\n" + "="*70)

print(f"\nPhysical Interpretation:")
print(f"  Material Type: {'Fluid-like (fast relaxation)' if tau_modular < 0.1 else 'Viscoelastic' if tau_modular < 100 else 'Solid-like'}")
print(f"  Relaxation Time: {tau_modular:.6f} s (time to decay to 37% of initial stress)")
print(f"  Initial Stiffness: {G0_modular:.2e} Pa ({G0_modular/1e3:.1f} kPa)")

## Key Takeaways

### Main Concepts

1. **Two API Approaches:**
   - **Pipeline API**: Fluent interface for rapid workflows (`Pipeline().load().fit().plot()`)
   - **Modular API**: Direct model control for customization (`Maxwell().fit()`)
   - Both produce identical numerical results

2. **NLSQ Optimization:**
   - Default backend provides 5-270x speedup vs SciPy
   - JAX JIT compilation + automatic differentiation
   - GPU acceleration available (additional 10-100x for large datasets)
   - Float64 precision enforced via safe_import_jax()

3. **Bayesian Uncertainty Quantification:**
   - Two-stage workflow: NLSQ (fast) → NUTS (warm-start)
   - Warm-start reduces convergence time 2-5x
   - Provides credible intervals and parameter correlations
   - Essential for poorly-constrained parameters

4. **ArviZ Diagnostic Suite:**
   - **6 essential plots** assess MCMC quality comprehensively
   - Must check: R-hat < 1.01, ESS > 400, divergences < 1%
   - Rank plot is most sensitive convergence diagnostic
   - Pair plot reveals parameter correlations and divergences

### When to Use Maxwell Model

**Appropriate for:**
- ✓ Polymer melts and solutions with single relaxation time
- ✓ Small strain linear viscoelastic regime
- ✓ Rapid screening of relaxation behavior
- ✓ Materials with predominantly viscous character

**Consider alternatives for:**
- ✗ Multiple relaxation times → Generalized Maxwell (Prony series)
- ✗ Non-exponential decay → Fractional Maxwell
- ✗ Finite equilibrium modulus → Zener model
- ✗ Solid-like materials → Kelvin-Voigt or Burgers model

### Common Pitfalls

1. **Float64 Precision:**
   - Always use `safe_import_jax()` pattern
   - NLSQ requires float64 for numerical stability
   - Automatic via package but verify in custom code

2. **Bayesian Convergence:**
   - **Never skip diagnostic checks** - R-hat, ESS, divergences
   - Use warm-start from NLSQ (essential for complex models)
   - Multiple chains (num_chains=4) recommended for production

3. **Model Selection:**
   - Check residuals for systematic trends
   - Maxwell often insufficient for real materials
   - Use Bayesian model comparison (WAIC, LOO) to compare alternatives

4. **Parameter Bounds:**
   - Set physically reasonable bounds in ParameterSet
   - Prevents optimization from exploring unphysical regions
   - Transforms to priors in Bayesian inference

## Further Reading

### Related Models
- [**Maxwell Model Handbook**](../../docs/source/models/classical/maxwell.rst) — Comprehensive theoretical foundation, experimental design, fitting guidance, and material interpretation
- [**Zener (Standard Linear Solid)**](02-zener-fitting.ipynb) — Adds parallel spring for finite equilibrium modulus
- [**SpringPot (Fractional Element)**](03-springpot-fitting.ipynb) — Power-law relaxation for broad spectra
- [**Generalized Maxwell**](../../docs/source/models/multi_mode/generalized_maxwell.rst) — Multiple relaxation times via Prony series

### Key References
- **Ferry, J.D. (1980)**. *Viscoelastic Properties of Polymers*, 3rd ed. Wiley. — Canonical reference for linear viscoelasticity and time-temperature superposition
- **Macosko, C.W. (1994)**. *Rheology: Principles, Measurements, and Applications*. Wiley-VCH. — Practical guide balancing theory and applications
- **Doi, M. & Edwards, S.F. (1986)**. *The Theory of Polymer Dynamics*. Oxford. — Connection to reptation theory and molecular dynamics

### Advanced Workflows
- [**Bayesian Basics**](../bayesian/01-bayesian-basics.ipynb) — Comprehensive NLSQ→NUTS workflow with detailed diagnostics
- [**Convergence Diagnostics**](../bayesian/03-convergence-diagnostics.ipynb) — Deep dive into all 6 ArviZ plots
- [**Mastercurve Generation**](../transforms/02-mastercurve-generation.ipynb) — Time-temperature superposition
- [**Multi-Technique Fitting**](../advanced/01-multi-technique-fitting.ipynb) — Constrained fitting across test modes

## Next Steps

**Continue the Basic Series:**
1. **[02-zener-fitting.ipynb](02-zener-fitting.ipynb)** — Finite equilibrium modulus (viscoelastic solid)
2. **[03-springpot-fitting.ipynb](03-springpot-fitting.ipynb)** — Fractional element with power-law relaxation
3. **[04-bingham-fitting.ipynb](04-bingham-fitting.ipynb)** — Yield stress fluids
4. **[05-power-law-fitting.ipynb](05-power-law-fitting.ipynb)** — Shear-thinning/thickening flow behavior

---

## Session Information

In [ ]:
# Print session information for reproducibility
import sys

import arviz as az
import rheojax

print(f"Python version: {sys.version}")
print(f"Rheo version: {rheojax.__version__}")
print(f"JAX version: {jax.__version__}")
print(f"JAX devices: {jax.devices()}")
print(f"NumPy version: {np.__version__}")
print(f"ArviZ version: {az.__version__}")